The aim of this notebook is to show how to use awscli together with the boto3 Python library to do some basic management of AWS EC2 instances. A lot of these code snippets can be turned into python scripts that can be hooked up with further shell commands to get to simple interactions like  
`aws-ssh MyJupyterInstance` 

Necessary setup:
* Python3 (I recommend Anaconda, #https://www.anaconda.com/download/)
* An AWS account, https://aws.amazon.com/
* AWS Access Key ID and Secret Access Key. You obtain these when creating a user in the AWS IAM
* An instance created via the EC2 dashboard

Install the python libraries:
  
`  pip install awscli`  
`  pip install boto3`

From the shell/command line, call  
  
`aws configure`  
  
This will ask you for the Access Key and Secret Access Key, as well as your default region.

Now that we have the setup and configuration out of the way, let's get started with some code! First, let's import boto

In [ ]:
import boto3

The main boto3 class we will be working with for the basic EC2 operations is the boto3 client, which gives low-level access to the AWS CLI. The full documentation can be found here: http://boto3.readthedocs.io/en/latest/reference/core/session.html#boto3.session.Session.client, but for now we need at most one of the extra arguments, *region_name*. This defaults to the region that you gave when calling `aws configure`, so can be safely ignored if you only have instances that you want to manage in this particular region. Let's assume that you do have instances spread across different regions, say the US West Coast and Western Europe. Let's connect to the Oregon data centre for now:

In [ ]:
ec2 = boto3.client('ec2', region_name='us-west-2')

To get a list of all available region names, you could also drop the region_name, which will connect to your default, and then call `describe_regions()`:

In [ ]:
regions = [x['RegionName'] for x in boto3.client('ec2').describe_regions()['Regions']]
regions

To obtain a list of your instances in the current region, simply call `describe_instances()`. The response will be in JSON format, and we can parse the relevant part with a simple list comprehension.

In [ ]:
response = ec2.describe_instances()

instances = [x for r in response['Reservations'] for x in r['Instances'] ]

So far so good. However, the response object contains all kinds of data, most of which we don't really need right now:

In [ ]:
instances[0]

Let's pick out the few relevant pieces of information and put the whole thing into a nested dictionary keyed on the instance name:

In [ ]:
keys = ['InstanceId', 'InstanceType', 'State']
instance_info = {instance['Tags'][0]['Value']: { k: instance[k] for k in keys } for instance in instances}

In [ ]:
instance_info